In [3]:
import gp, gp.data
import pandas as pd
from glob import glob
import re
import os
import time
#import wget
import getpass
from io import StringIO

#enter your GP info early to access DiSCoVER
print("Welcome to the DataFrame Generator\n")
username = input("Enter GP username:")
password = getpass.getpass("Enter GP password:")

#access the GP server, import the select .gct file to parse  NOTE: needs to be revised
gpserver = gp.GPServer('https://cloud.genepattern.org/gp', username, password)
print("Select the GCT file to process:")
df = pd.read_csv('/home/jovyan/df_brca_fpkm.gct', header=2,sep='\t').drop('Description',axis=1)
#df = pd.read_csv('https://datasets.genepattern.org/data/ccmi_tutorial/2018-09-20/BRCA_HUGO_symbols.preprocessed.gct', header=2,sep='\t').drop('Description',axis=1)

names = df.drop('NAME',axis=1).columns

#this will get the discover module loading   
module = gp.GPTask(gpserver, "DiSCoVER")
module.param_load()

csv_wave1_folder = "CSVraw3"
csv_wave2_folder = "CSVresults3"

#directory for the 1st wave of CSV files
print("a folder named", csv_wave1_folder," will now be added to your directory")
print("the resulting CSV files will be added to", csv_wave1_folder)
dir = "/home/jovyan/"+csv_wave1_folder
try:
  print("a folder named", csv_wave1_folder," will now be added to your directory \n")
  os.mkdir(dir)
except:
  pass

#if this directory is empty, files will be added
if len(os.listdir(dir) ) == 0:
    print("Directory is empty \n")
    for name in names:
        print("adding files")
        df_new = df[['NAME',name]].to_csv("/home/jovyan/"+csv_wave1_folder+"/"+name+'raw.csv',header=True,index=True)
else:    
    print("Directory is not empty")
    
#access and sort all the wave1 CSV files
cancer_and_normal = glob(csv_wave1_folder+"/*.csv")
cancer_and_normal.sort()

# create the directory fot the wave2 CSV results
dir = "/home/jovyan/"+csv_wave2_folder
yoit = dir
try:
    os.mkdir(dir)
    print("Creating "+csv_wave2_folder+" folder \n")
except:
    pass
        
#current number of items in CSVresults    
number_one = (len(os.listdir(yoit)))*2
number_two = number_one/2
print("Items in,", csv_wave2_folder,": ", number_two)


#this will access every two files that have a certain        
for files in cancer_and_normal:
    if (number_one>0):
        number_one = number_one-1
        print('Skipping one file in '+csv_wave1_folder)
    else:
        
        directory = str (files)
        #namecell = (directory[7:22:1])
        
        namecell =(directory[8:23:1])
        print(namecell)
        #typecell = (directory[21:23:1])
        typecell = (directory[21:23:1])
        print(typecell)
        print(files)

        if typecell == "01" :
            cancer_sample_name = namecell
            cancer_sample_dir = directory

        elif typecell == "11" :
            normal_sample_name = namecell
            normal_sample_dir = directory
            
            job_spec = module.make_job_spec()
            uploaded_gene_file = gpserver.upload_file(cancer_sample_name, cancer_sample_dir)
            print(uploaded_gene_file)
            uploaded_control_file = gpserver.upload_file(normal_sample_name, normal_sample_dir )
            job_spec.set_parameter("gene_expression", uploaded_gene_file.get_url()) 
            job_spec.set_parameter("use_custom_control", "True")
            job_spec.set_parameter("control", uploaded_control_file.get_url())

            job = gpserver.run_job(job_spec, False)
            job.is_finished()  # Quaries the server and returns True if the job is complete, False otherwise
            job.get_info()  # Returns a brief description of the job's current state

            while not job.is_finished():
                print("Still running...")
                time.sleep(120)
            number_two = number_two+1
            print("CSV Wave2 File Created: ", number_two)

            print(job.is_finished())
            output_list = job.get_output_files()  # Get a list of output files

            for file in output_list:  # Loop through each output file
                if file.get_url().endswith('.txt'):
                    print('skipping',file)
                elif file.get_url().endswith('.csv'):
                    print( file.get_url() )  # Print the URL to the file
                    
        #             GP_csv = file.get_url()
        #             temp = GP_csv.split('/')
        #             # programatically access that job to open the file
        #             GP_csv = eval('job'+temp[5]+'.get_file("'+temp[6]+'")')
        #             print('job'+temp[5]+'.get_file("'+temp[6]+'")')
        
                    drug_df = pd.read_csv(StringIO(file.read()),index_col=0)
                    print(drug_df.head())

                    #if (len(os.listdir(dir) ) != len(cancer_and_normal)/2):
                    print('==> About to write file',dir+cancer_sample_name+'_DiSCoVER_results.csv')
                    drug_df.to_csv(dir+'/'+cancer_sample_name+'_DiSCoVER_results.csv')
                else:
                    print('Unknown file type:',file)

    #             wget.download(file,'/home/jovyan/CSVresults')
                # if the folder has the necessary number of files...
        
#access and sort all of the wave2 csv files
second_wave = glob(csv_wave2_folder+"/*.csv")
second_wave.sort()
        
#this is where the discover outputs will be turned into a dataframe and gct file        
#numFiles carries discover output csv files
numFiles = []
#all_paths = glob("CSVresults/*.csv")
#all_paths

#df_preList carries dataframes
df_preList = []
score_df = pd.DataFrame([])

#this for loop will fill up the list numFiles with all the CSV files
for filename in second_wave:
    numFiles.append(filename)
    
# this for loop will take every CSV in the list numFiles and create their corresponding dataframe
# these dataframes are then added to the list df_preList
for i in range(len(second_wave)):
        df_csv = pd.read_csv(numFiles[i])
        df_preList.append(df_csv)
        
#this for loop will extract the "score" column from every csv dataframe and 
#add them to an existing one that only carries name, moa, and the first score
df_final = pd.DataFrame()
counter = 0
for i in range(len(second_wave)):
    if counter == 0:
        df_final = df_preList[counter]
        df_final = df_final.set_index("drug")
        df_final.index.names = ['NAME']
        tempstring = str (numFiles[i])
        tempstring =(tempstring[17:32:1])
        
        new_order = ['moa','score']
        df_final=df_final[new_order]
        
        df_final=df_final.rename(columns = {'score':tempstring,'moa':'Description'})
        
    else:
        tempstring = str (numFiles[i])
        tempstring =(tempstring[17:32:1])
        col_name = tempstring
        temp = df_preList[i]
        temp = temp.set_index("drug")
        df_final[col_name] = temp["score"]
        
    counter += 1
    
def standarize_string(what):
    ascii_8bit = re.sub(r'[^ -~]', '', what).lower() # keeping only 8-bit ascii, making lowercase (http://www.catonmat.net/blog/my-favorite-regex/)
    return ascii_8bit
    #return re.sub(r'[ _-]', '', ascii_8bit) # remove space, underscore, and dash (" _-")
    
for index, row in df_final.iterrows():
    
    df_final.loc[index,'Description'] = standarize_string(str (row['Description']))
    
df_final.index.names = ['NAME']    
df_final = df_final.sort_values(by=['NAME'])

output_name = "df_fpkm"

gp.data.write_gct(df_final, file_path='./'+output_name+'.gct')
print("DataFrame Successfully Created! Please look for " +output_name+ " in your home directory")

            

Welcome to the DataFrame Generator

Enter GP username:HugoElcabezas
Enter GP password:········
Select the GCT file to process:


KeyError: "['NAME'] not found in axis"

In [14]:
gp.data.write_gct(df_final, file_path='./'+output_name+'.gct')
print("DataFrame Successfully Created! Please look for " +output_name+ " in your home directory")


DataFrame Successfully Created! Please look for df_fpkm in your home directory
